<a href="https://colab.research.google.com/github/J-Pitts/J-Pitts.github.io/blob/master/CensusData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install censusdata

In [2]:
import pandas as pd
import censusdata
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.precision', 2)

Get Relevant Information

In [3]:
#EMPLOYMENT STATUS FOR THE POPULATION = B23025
employment_pop = (pd.DataFrame.from_dict(dict(censusdata.censustable('acs5', 2015, 'B23025')))).columns.values.tolist()
#B23025_001E	Estimate!!Total

#EDUCATIONAL ATTAINMENT FOR THE POPULATION
education_pop = (pd.DataFrame.from_dict(dict(censusdata.censustable('acs5', 2015, 'B15003')))).columns.values.tolist()

#GEOGRAPHIC SPECIFICATION
all_states_all_counties = censusdata.censusgeo([('state', '*'), ('county', '*')])

#WORKFORCE SIZE
workforce_size = (pd.DataFrame.from_dict(dict(censusdata.censustable('acs5', 2015, 'C24010')))).columns.values.tolist()
#C24010_001E SEX BY OCCUPATION FOR THE CIVILIAN EMPLOYED POPULATION 16 YEARS AND OVER

#GEOGRAPHIC MOBILITY
geographic_mobility = (pd.DataFrame.from_dict(dict(censusdata.censustable('acs5', 2015, 'B07410')))).columns.values.tolist()
#B07410_003E Estimate!!Total living in area 1 year ago!!With income

#Business Bachelors Degrees
business_bachelors = (pd.DataFrame.from_dict(dict(censusdata.censustable('acs5', 2015, 'C15010')))).columns.values.tolist()
#C15010_004E	Estimate!!Total!!Business


In [4]:
df = censusdata.download('acs5', 2015,
                             all_states_all_counties,
                             employment_pop+education_pop+workforce_size+geographic_mobility+business_bachelors)

cols_dict = {'B23025_001E':	'EMPLOYMENT_POPULATION',
             'B15003_001E': 'EDUCATION_POPULATION',
             'B15003_023E': 'EDUCATION_Masters',
             'B15003_022E': 'EDUCATION_Bachelors',
             'C24010_001E': 'CIVILIAN_Workforce',
             'B07410_003E':	'GEOGRAPHICAL_MOBILITY_Workforce',
             'C15010_004E':	'EDUCATION_BUSINESS_BACHELORS',
             'B23025_005E':	'Civilian_Unemployed',
            }


In [ ]:
df = df.rename(columns=cols_dict)
df_main = df[cols_dict.values()]

x = df.index.to_series().str.split()

location = x.to_string()

counties =[]
states = []

x = (location.split('\n'))
for i in range(len(x)):
    y= x[i].split(':')[0]
    counties.append( y.split(',')[0])
    states.append( y.split(',')[1])
  
df_main['counties'] = counties
df_main['state'] = states
df_main = df_main.reset_index(drop=True)

In [7]:
master_bystate = df_main.groupby(["state"])["EDUCATION_Masters"].sum()
total_bystate = df_main.groupby(["state"])["EDUCATION_POPULATION"].sum()

education_df = pd.DataFrame(data =[master_bystate.values.tolist(), total_bystate.values.tolist()]).transpose()
education_df = education_df.set_index(pd.unique(df_main['state']))
education_df = education_df.rename(columns={0: 'EDUCATION_Masters', 1: 'EDUCATION_POPULATION'})
education_df = education_df.rename_axis('State')
print(education_df['EDUCATION_Masters']/education_df['EDUCATION_POPULATION'])

State
 Texas                   0.06
 Vermont                 0.07
 Puerto Rico             0.07
 Pennsylvania            0.05
 Missouri                0.08
 Montana                 0.10
 Nebraska                0.12
 Florida                 0.09
 Georgia                 0.19
 Maine                   0.07
 Maryland                0.08
 Massachusetts           0.07
 Michigan                0.06
 Minnesota               0.09
 Nevada                  0.06
 New Hampshire           0.06
 New Jersey              0.08
 New Mexico              0.07
 Mississippi             0.05
 New York                0.07
 North Carolina          0.12
 North Dakota            0.12
 Ohio                    0.08
 Tennessee               0.08
 Oklahoma                0.06
 Oregon                  0.07
 Rhode Island            0.07
 South Carolina          0.07
 Iowa                    0.05
 Kansas                  0.10
 Kentucky                0.10
 Colorado                0.08
 Delaware                0.11
 Dis

In [9]:

bachelors_ranking = df_main['EDUCATION_Bachelors'].values.sum() / df_main['EDUCATION_POPULATION'].values.sum()
print('Percent of population with a bachelors degree nationwide: {:.2f}%'.format(bachelors_ranking*100))

masters_ranking = df_main['EDUCATION_Masters'].values.sum() / df_main['EDUCATION_POPULATION'].values.sum()
print('\nPercent of population with a masters degree nationwide: {:.2f}%'.format(masters_ranking*100))

business_bachelors_ranking = df_main['EDUCATION_BUSINESS_BACHELORS'].values.sum() / df_main['EDUCATION_POPULATION'].values.sum()
print('\nPercent of population with a business bachelors degree nationwide: {:.2f}%'.format(business_bachelors_ranking *100))


Percent of population with a bachelors degree nationwide: 18.51%

Percent of population with a masters degree nationwide: 7.93%

Percent of population with a business bachelors degree nationwide: 6.11%
